In [ ]:
# Environment Detection
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f'Environment: {"Colab" if IN_COLAB else "Local"}')


In [ ]:
# 🔧 Environment Detection and Setup
import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
env_label = 'Google Colab' if IN_COLAB else 'Local'
print(f'Environment: {env_label}')

# Setup environment-specific configurations
if IN_COLAB:
    print('📝 Colab-specific optimizations enabled')
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except Exception:
        pass


## API Keys and .env Files\n\nMany providers require API keys. Do not hardcode secrets in notebooks. Use a local .env file that the notebook loads at runtime.\n\n- Why .env? Keeps secrets out of source control and tutorials.\n- Where? Place `.env.local` (preferred) or `.env` in the same folder as this notebook. `.env.local` overrides `.env`.\n- What keys? Common: `POE_API_KEY` (Poe-compatible servers), `OPENAI_API_KEY` (OpenAI-compatible), `HF_TOKEN` (Hugging Face).\n- Find your keys:\n  - Poe-compatible providers: see your provider's dashboard for an API key.\n  - Hugging Face: create a token at https://huggingface.co/settings/tokens (read scope is usually enough).\n  - Local servers: you may not need a key; set `OPENAI_BASE_URL` instead (e.g., http://localhost:1234/v1).\n\nThe next cell will: load `.env.local`/`.env`, prompt for missing keys, and optionally write `.env.local` with secure permissions so future runs just work.

In [ ]:
# 🔐 Load and manage secrets from .env\n# This cell will: (1) load .env.local/.env, (2) prompt for missing keys, (3) optionally write .env.local (0600).\n# Location: place your .env files next to this notebook (recommended) or at project root.\n# Disable writing: set SAVE_TO_ENV = False below.\nimport os, pathlib\nfrom getpass import getpass\n\n# Install python-dotenv if missing\ntry:\n    import dotenv  # type: ignore\nexcept Exception:\n    import sys, subprocess\n    if 'IN_COLAB' in globals() and IN_COLAB:\n        try:\n            import IPython\n            ip = IPython.get_ipython()\n            if ip is not None:\n                ip.run_line_magic('pip', 'install -q python-dotenv>=1.0.0')\n            else:\n                subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'python-dotenv>=1.0.0'])\n        except Exception as colab_exc:\n            print('⚠️ Colab pip fallback failed:', colab_exc)\n            raise\n    else:\n        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'python-dotenv>=1.0.0'])\n    import dotenv  # type: ignore\n\n# Prefer .env.local over .env\ncwd = pathlib.Path.cwd()\nenv_local = cwd / '.env.local'\nenv_file = cwd / '.env'\nchosen = env_local if env_local.exists() else (env_file if env_file.exists() else None)\nif chosen:\n    dotenv.load_dotenv(dotenv_path=str(chosen))\n    print(f'Loaded env from {chosen.name}')\nelse:\n    print('No .env.local or .env found; will prompt for keys.')\n\n# Keys we might use in this notebook\nkeys = ['POE_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']\nmissing = [k for k in keys if not os.environ.get(k)]\nfor k in missing:\n    val = getpass(f'Enter {k} (hidden, press Enter to skip): ')\n    if val:\n        os.environ[k] = val\n\n# Decide whether to persist to .env.local for convenience\nSAVE_TO_ENV = True  # set False to disable writing\nif SAVE_TO_ENV:\n    target = env_local\n    existing = {}\n    if target.exists():\n        try:\n            for line in target.read_text().splitlines():\n                if not line.strip() or line.strip().startswith('#') or '=' not in line:\n                    continue\n                k,v = line.split('=',1)\n                existing[k.strip()] = v.strip()\n        except Exception:\n            pass\n    for k in keys:\n        v = os.environ.get(k)\n        if v:\n            existing[k] = v\n    lines = []\n    for k,v in existing.items():\n        # Always quote; escape backslashes and double quotes for safety\n        escaped = v.replace("\\", "\\\\")\n        escaped = escaped.replace("\"", "\\"")\n        vv = f'"{escaped}"'\n        lines.append(f"{k}={vv}")\n    target.write_text('\\n'.join(lines) + '\\n')\n    try:\n        target.chmod(0o600)  # 600\n    except Exception:\n        pass\n    print(f'🔏 Wrote secrets to {target.name} (permissions 600)')\n\n# Simple recap (masked)\ndef mask(v):\n    if not v: return '∅'\n    return v[:3] + '…' + v[-2:] if len(v) > 6 else '•••'\nfor k in keys:\n    print(f'{k}:', mask(os.environ.get(k)))\n

In [ ]:
# 🌐 ALAIN Provider Setup (Poe/OpenAI-compatible)
# About keys: If you have POE_API_KEY, this cell maps it to OPENAI_API_KEY and sets OPENAI_BASE_URL to Poe.
# Otherwise, set OPENAI_API_KEY (and optionally OPENAI_BASE_URL for local/self-hosted servers).
import os
try:
    # Prefer Poe; fall back to OPENAI_API_KEY if set
    poe = os.environ.get('POE_API_KEY')
    if poe:
        os.environ.setdefault('OPENAI_BASE_URL', 'https://api.poe.com/v1')
        os.environ.setdefault('OPENAI_API_KEY', poe)
    # Prompt if no key present
    if not os.environ.get('OPENAI_API_KEY'):
        from getpass import getpass
        os.environ['OPENAI_API_KEY'] = getpass('Enter POE_API_KEY (input hidden): ')
        os.environ.setdefault('OPENAI_BASE_URL', 'https://api.poe.com/v1')
    # Ensure openai client is installed
    try:
        from openai import OpenAI  # type: ignore
    except Exception:
        import sys, subprocess
        if 'IN_COLAB' in globals() and IN_COLAB:
            try:
                import IPython
                ip = IPython.get_ipython()
                if ip is not None:
                    ip.run_line_magic('pip', 'install -q openai>=1.34.0')
                else:
                    cmd = [sys.executable, "-m", "pip", "install", '-q', 'openai>=1.34.0']
                    try:
                        subprocess.check_call(cmd)
                    except Exception as exc:
                        if IN_COLAB:
                            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
                            if packages:
                                try:
                                    import IPython
                                    ip = IPython.get_ipython()
                                    if ip is not None:
                                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                                    else:
                                        import subprocess as _subprocess
                                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                                except Exception as colab_exc:
                                    print('⚠️ Colab pip fallback failed:', colab_exc)
                                    raise
                            else:
                                print('No packages specified for pip install; skipping fallback')
                        else:
                            raise
            except Exception as colab_exc:
                print('⚠️ Colab pip fallback failed:', colab_exc)
                raise
        else:
            cmd = [sys.executable, "-m", "pip", "install", '-q', 'openai>=1.34.0']
            try:
                subprocess.check_call(cmd)
            except Exception as exc:
                if IN_COLAB:
                    packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
                    if packages:
                        try:
                            import IPython
                            ip = IPython.get_ipython()
                            if ip is not None:
                                ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                            else:
                                import subprocess as _subprocess
                                _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                        except Exception as colab_exc:
                            print('⚠️ Colab pip fallback failed:', colab_exc)
                            raise
                    else:
                        print('No packages specified for pip install; skipping fallback')
                else:
                    raise
        from openai import OpenAI  # type: ignore
    # Create client
    from openai import OpenAI
    client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
    print('✅ Provider ready:', os.environ.get('OPENAI_BASE_URL'))
except Exception as e:
    print('⚠️ Provider setup failed:', e)


In [ ]:
# 🔎 Provider Smoke Test (1-token)
import os
model = os.environ.get('ALAIN_MODEL') or 'gpt-4o-mini'
if 'client' not in globals():
    print('⚠️ Provider client not available; skipping smoke test')
else:
    try:
        resp = client.chat.completions.create(model=model, messages=[{"role":"user","content":"ping"}], max_tokens=1)
        print('✅ Smoke OK:', resp.choices[0].message.content)
    except Exception as e:
        print('⚠️ Smoke test failed:', e)


> Generated by ALAIN (Applied Learning AI Notebooks) — 2025-09-16.


# Deep Dive into GPT‑Oss‑20B: Architecture, Training, and Deployment

This notebook guides advanced practitioners through the intricacies of GPT‑Oss‑20B, covering its transformer architecture, tokenization, data pipelines, fine‑tuning strategies, distributed training, evaluation, inference optimization, deployment, and ethical considerations. It balances theory with hands‑on code, enabling researchers to replicate, extend, and responsibly deploy the model.


> ⏱️ Estimated time to complete: 36–60 minutes (rough).  
> 🕒 Created (UTC): 2025-09-16T02:59:29.868Z



## Learning Objectives

By the end of this tutorial, you will be able to:

1. Explain the architectural components and design choices of GPT‑Oss‑20B.
2. Demonstrate how to prepare data, fine‑tune, and apply parameter‑efficient tuning techniques.
3. Illustrate distributed training with DeepSpeed and inference optimization for production.
4. Critically assess ethical implications and propose bias mitigation strategies.


## Prerequisites

- Python 3.10+
- PyTorch 2.0+
- Hugging Face Transformers 4.35+
- Basic knowledge of transformer models and deep learning workflows


## Setup

Let's install the required packages and set up our environment.


In [ ]:
# Install packages (Colab-compatible)
# Check if we're in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q ipywidgets>=8.0.0 torch>=2.0 transformers>=4.35 accelerate datasets deepspeed fastapi torchserve
else:
    import subprocess
    cmd = [sys.executable, "-m", "pip", "install"] + ["ipywidgets>=8.0.0","torch>=2.0","transformers>=4.35","accelerate","datasets","deepspeed","fastapi","torchserve"]
    try:
        subprocess.check_call(cmd)
    except Exception as exc:
        if IN_COLAB:
            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
            if packages:
                try:
                    import IPython
                    ip = IPython.get_ipython()
                    if ip is not None:
                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                    else:
                        import subprocess as _subprocess
                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                except Exception as colab_exc:
                    print('⚠️ Colab pip fallback failed:', colab_exc)
                    raise
            else:
                print('No packages specified for pip install; skipping fallback')
        else:
            raise

print('✅ Packages installed!')

In [ ]:
# Ensure ipywidgets is installed for interactive MCQs
try:
    import ipywidgets  # type: ignore
    print('ipywidgets available')
except Exception:
    import sys, subprocess
    cmd = [sys.executable, "-m", "pip", "install", '-q', 'ipywidgets>=8.0.0']
    try:
        subprocess.check_call(cmd)
    except Exception as exc:
        if IN_COLAB:
            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
            if packages:
                try:
                    import IPython
                    ip = IPython.get_ipython()
                    if ip is not None:
                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                    else:
                        import subprocess as _subprocess
                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                except Exception as colab_exc:
                    print('⚠️ Colab pip fallback failed:', colab_exc)
                    raise
            else:
                print('No packages specified for pip install; skipping fallback')
        else:
            raise


## Step 1: Environment Setup and Model Loading

Before we can play with GPT‑Oss‑20B, we need a clean playground where all the tools live. Think of it like setting up a kitchen: you need a stove, a fridge, a cutting board, and the right utensils. In the machine‑learning world, those utensils are libraries such as **PyTorch**, **Transformers**, **Accelerate**, **Datasets**, **DeepSpeed**, **FastAPI**, **TorchServe**, and **ipywidgets**. We’ll install them with `pip`, set an environment variable for Hugging Face cache, and verify that everything is the right version.

### Why this order?
- **`pip install`** first: ensures we have the latest compatible packages.
- **Version check**: a quick sanity test that the installed packages match the notebook’s expectations.
- **Model loading**: pulls GPT‑Oss‑20B from Hugging Face Hub and prepares the tokenizer.

### Key terms explained
- **Environment variable (`HF_HOME`)**: a folder where Hugging Face stores cached models and tokenizers. Setting it keeps your cache tidy and prevents accidental downloads.
- **PyTorch**: the deep‑learning framework that powers the model’s tensors and GPU acceleration.
- **Transformers**: the library that provides the GPT‑Oss‑20B architecture and tokenizer.
- **Accelerate**: a helper that abstracts device placement (CPU/GPU/TPU) and distributed training.
- **Datasets**: a fast data loading library that handles large text corpora.
- **DeepSpeed**: a library for efficient large‑model training (memory optimization, mixed‑precision, etc.).
- **FastAPI**: a lightweight web framework for building REST APIs.
- **TorchServe**: a production‑ready model serving tool.
- **ipywidgets**: interactive widgets for Jupyter notebooks.
- **Seed**: a number that initializes random number generators to make experiments reproducible.

### Trade‑offs
- **`pip` vs. `conda`**: `pip` gives you the latest releases but may require manual CUDA setup; `conda` bundles CUDA but can lag behind.
- **TorchServe vs. FastAPI**: TorchServe is opinionated and optimized for serving, while FastAPI offers more flexibility for custom endpoints.
- **Local vs. cloud**: Running locally keeps data private but may hit GPU memory limits; cloud instances provide more resources but add cost.

With this foundation, we’re ready to pull GPT‑Oss‑20B into memory and start experimenting.



In [ ]:
# Cell 1: Install required packages
# This cell uses subprocess to run pip commands.
# It will silently ignore errors if a package is already installed.
import subprocess, sys

packages = [
    "torch>=2.0",
    "transformers>=4.35",
    "accelerate",
    "datasets",
    "deepspeed",
    "fastapi",
    "torchserve",
    "ipywidgets>=8.0.0"
]

for pkg in packages:
    try:
        cmd = [sys.executable, "-m", "pip", "install", pkg]
        try:
            subprocess.check_call(cmd)
        except Exception as exc:
            if IN_COLAB:
                packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
                if packages:
                    try:
                        import IPython
                        ip = IPython.get_ipython()
                        if ip is not None:
                            ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                        else:
                            import subprocess as _subprocess
                            _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                    except Exception as colab_exc:
                        print('⚠️ Colab pip fallback failed:', colab_exc)
                        raise
                else:
                    print('No packages specified for pip install; skipping fallback')
            else:
                raise
    except subprocess.CalledProcessError as e:
        print(f"Warning: failed to install {pkg}. Continuing...", file=sys.stderr)

print("Package installation complete.")


In [ ]:
# Cell 2: Verify installed versions
import torch, transformers, accelerate, datasets, deepspeed, fastapi, torchserve, ipywidgets

print("PyTorch version:", torch.__version__)
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)
print("Datasets version:", datasets.__version__)
print("DeepSpeed version:", deepspeed.__version__)
print("FastAPI version:", fastapi.__version__)
print("TorchServe version:", torchserve.__version__)
print("ipywidgets version:", ipywidgets.__version__)



In [ ]:
# Cell 3: Set environment variable and load GPT‑Oss‑20B
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1️⃣ Set HF_HOME to keep cache tidy
os.environ["HF_HOME"] = "/tmp/hf_cache"
print("HF_HOME set to", os.environ["HF_HOME"])

# 2️⃣ Reproducibility seed
SEED = 42
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
print("Random seed set to", SEED)

# 3️⃣ Load tokenizer and model (this may download the 20B weights if not cached)
MODEL_NAME = "gpt-oss-20b"
print(f"Loading tokenizer for {MODEL_NAME}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
except Exception as e:
    print("Error loading tokenizer:", e)
    raise

print(f"Loading model for {MODEL_NAME} (this may take a while)...")
try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,  # use FP16 for memory efficiency
        device_map="cuda:0" if torch.cuda.is_available() else "cpu"          # automatically place layers on available GPUs
    )
except Exception as e:
    print("Error loading model:", e)
    raise

print("Model and tokenizer loaded successfully.")


## Step 2: GPT‑Oss‑20B Architecture Overview

Imagine a gigantic factory that can read a sentence, understand it, and then write a continuation. Each station in this factory is a *transformer layer* that takes the raw text, turns it into a set of numbers (embeddings), lets the numbers talk to each other through *self‑attention*, and then refines the result with a small neural network (the *feed‑forward* block). The factory is built on a *stack* of these layers—20 B‑GPT‑Oss has 32 such layers, each with 32 attention heads and a hidden dimension of 8 192.

### Why this design?
- **Depth (32 layers)**: More layers let the model learn increasingly abstract patterns, like how a sentence can be broken down into syntax, semantics, and world knowledge.
- **Width (32 heads, 8 192 hidden size)**: Wider layers allow the model to capture a richer set of relationships between tokens.
- **Self‑attention**: Every token can directly look at every other token, which is essential for long‑context reasoning.
- **LayerNorm + Residuals**: These keep gradients stable and help the network learn faster.
- **Positional Encoding**: Since transformers have no inherent sense of order, positional embeddings give each token a unique “address” in the sequence.

### Key terms explained
- **Transformer**: A neural architecture that relies on self‑attention to process sequences.
- **Self‑attention**: A mechanism where each token computes a weighted sum of all tokens, allowing global context.
- **Feed‑forward network (FFN)**: A two‑layer MLP applied to each token independently.
- **LayerNorm**: Normalizes activations across the hidden dimension to stabilize training.
- **Residual connection**: Adds the input of a sub‑module to its output, helping gradients flow.
- **Positional embedding**: A learned vector added to token embeddings to encode position.
- **ZeRO**: A memory‑optimization technique used during training (not in inference).

### Rationale & trade‑offs
The 20 B parameter count is a sweet spot between *expressive power* and *resource feasibility*. A larger model can capture more nuanced language patterns but requires more GPU memory and longer inference times. The chosen depth/width balance ensures that each layer can learn complex interactions without exploding memory usage. However, this also means that inference on a single GPU can be slow; sharding or quantization is often necessary for production.

In the next step we’ll dive into how the model turns raw text into these embeddings—tokenization—and how we manage the vocabulary that feeds into the transformer.



In [ ]:
# Inspect GPT‑Oss‑20B configuration and a quick forward pass
# This cell is short (<30 lines) and includes error handling.
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set a reproducible seed
SEED = 1234
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

MODEL_NAME = "gpt-oss-20b"

# Load tokenizer and model (device_map="cuda:0" if torch.cuda.is_available() else "cpu" shards across GPUs if available)
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="cuda:0" if torch.cuda.is_available() else "cpu"
    )
except Exception as e:
    print("Failed to load model/tokenizer:", e)
    raise

# Print key hyperparameters
config = model.config
print("\n=== Model Hyperparameters ===")
print(f"Number of layers: {config.num_hidden_layers}")
print(f"Hidden size: {config.hidden_size}")
print(f"Attention heads: {config.num_attention_heads}")
print(f"Intermediate size (FFN): {config.intermediate_size}")
print(f"Vocabulary size: {config.vocab_size}")
print(f"Max sequence length: {config.max_position_embeddings}")

# Quick forward pass on a sample sentence
sample = "The quick brown fox jumps over the lazy dog"
inputs = tokenizer(sample, return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    print("\nLogits shape:", logits.shape)  # (batch, seq_len, vocab)

print("\n--- End of inspection ---")


## Step 3: Tokenization and Vocabulary Management

### 1️⃣ What is tokenization?
Think of a sentence as a long string of letters. A tokenizer is like a *smart cutting board* that slices this string into pieces (tokens) that the model can understand. For GPT‑Oss‑20B we use a **Byte‑Pair Encoding (BPE)** tokenizer, which learns the most common sub‑word units from the training data. The result is a *vocabulary* – a dictionary that maps each token to a unique integer ID.

### 2️⃣ Why BPE and not words?
- **Memory efficiency**: A full word‑level vocab for English would need >200k entries, blowing up the embedding matrix. BPE keeps the vocab around 50k–80k tokens.
- **Robustness to OOV**: Rare or unseen words are broken into known sub‑words, so the model can still process them.
- **Speed**: Tokenization is linear in the number of characters, and BPE tables are fast to look up.

### 3️⃣ Key terms
- **Tokenizer**: The software that turns raw text into token IDs.
- **Vocabulary (vocab)**: The mapping from token strings to integer IDs.
- **Special tokens**: Tokens like ```, ```, or `` that signal start/end of a sequence or padding.
- **Token IDs**: Integers that index into the model’s embedding matrix.
- **Embedding matrix**: A 2‑D tensor of shape `(vocab_size, hidden_size)` that converts token IDs into dense vectors.
- **`tokenizer.add_tokens()`**: A method to extend the vocab with new tokens.
- **`model.resize_token_embeddings()`**: Adjusts the embedding matrix to match the new vocab size.

### 4️⃣ Managing the vocabulary
1. **Inspect the current vocab** – you can print the size, list special tokens, and see how many tokens a sentence expands into.
2. **Add new tokens** – useful for domain‑specific terminology or control tokens.
3. **Resize embeddings** – after adding tokens you must enlarge the embedding matrix; otherwise the model will crash.
4. **Save and reload** – keep the updated tokenizer in a folder so you can reuse it.

### 5️⃣ Trade‑offs
| Aspect | Large vocab | Small vocab |
|--------|-------------|-------------|
| **Embedding size** | Larger matrix → more GPU memory | Smaller matrix → less memory |
| **Tokenization granularity** | Fewer sub‑word splits → longer tokens | More splits → shorter tokens |
| **Speed** | Slightly slower lookup due to larger table | Faster lookup |
| **Coverage** | Better coverage of rare words | More OOV tokens, more sub‑word splits |

Choosing the right vocab size is a balancing act: a bigger vocab reduces the number of tokens per sentence (good for memory‑bound inference) but increases the embedding matrix (bad for GPU memory). GPT‑Oss‑20B uses a 50k‑token BPE vocab, which is a sweet spot for English‑style text.

### 6️⃣ Next step preview
In the next section we’ll build a **data pipeline**: clean raw text, split it into training examples, and feed it into the tokenizer to create the datasets that will train or fine‑tune the model.



In [ ]:
# Cell 1: Load the GPT‑Oss‑20B tokenizer and inspect its vocabulary
# ---------------------------------------------------------------
import torch
from transformers import AutoTokenizer

# 1️⃣ Load the tokenizer (this may download the vocab if not cached)
MODEL_NAME = "gpt-oss-20b"
print(f"Loading tokenizer for {MODEL_NAME}…")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
except Exception as e:
    print("Error loading tokenizer:", e)
    raise

# 2️⃣ Basic stats
print("\n=== Tokenizer statistics ===")
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"Number of special tokens: {len(tokenizer.special_tokens_map)}")
print("Special tokens mapping:")
for key, value in tokenizer.special_tokens_map.items():
    print(f"  {key}: {value}")

# 3️⃣ Encode a sample sentence and show token IDs
sample = "The quick brown fox jumps over the lazy dog."
encoded = tokenizer(sample, return_tensors="pt")
print("\nEncoded token IDs:", encoded["input_ids"][0])
print("Number of tokens (including special tokens):", encoded["input_ids"][0].size(0))

# 4️⃣ Decode back to text to verify round‑trip
decoded = tokenizer.decode(encoded["input_ids"][0])
print("\nDecoded text:", decoded)



In [ ]:
# Cell 2: Adding new tokens and resizing the model’s embeddings
# ---------------------------------------------------------------
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1️⃣ Load tokenizer and model (device_map="cuda:0" if torch.cuda.is_available() else "cpu" shards across GPUs if available)
MODEL_NAME = "gpt-oss-20b"
print(f"\nLoading model for {MODEL_NAME}…")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="cuda:0" if torch.cuda.is_available() else "cpu"
    )
except Exception as e:
    print("Error loading model/tokenizer:", e)
    raise

# 2️⃣ Define new domain‑specific tokens
new_tokens = ["<BILL>", "<DATE>", "<ORG>"]
print("\nAdding new tokens to tokenizer:", new_tokens)
added = tokenizer.add_tokens(new_tokens)
print(f"Number of tokens added: {added}")

# 3️⃣ Resize the model’s embedding matrix to accommodate new tokens
model.resize_token_embeddings(len(tokenizer))
print("Resized embedding matrix to", len(tokenizer))

# 4️⃣ Verify that new tokens can be encoded
sample = "<BILL> 2023-08-15 <ORG>"
encoded = tokenizer(sample, return_tensors="pt")
print("\nEncoded IDs for new tokens:", encoded["input_ids"][0])

# 5️⃣ Optional: save the updated tokenizer for future reuse
save_dir = "./gpt-oss-20b-tokenizer"
print(f"\nSaving updated tokenizer to {save_dir}…")
try:
    tokenizer.save_pretrained(save_dir)
    print("Tokenizer saved successfully.")
except Exception as e:
    print("Failed to save tokenizer:", e)



## Step 4: Data Pipeline – Preprocessing and Dataset Construction

When you train a language model, the raw text you hand it is like a messy pile of LEGO bricks. The model needs a clean, well‑structured set of bricks that it can stack into sentences, paragraphs, and eventually entire books. This section walks through the *data pipeline* that turns raw text into a Hugging Face `Dataset` ready for fine‑tuning. We’ll cover:

1. **Data ingestion** – loading from local files or public datasets.
2. **Cleaning & filtering** – removing noise, normalizing whitespace, and enforcing a maximum sequence length.
3. **Tokenization** – converting text to token IDs with the GPT‑Oss‑20B tokenizer.
4. **Dataset construction** – shuffling, splitting, and caching for efficient training.
5. **Batching & collating** – preparing mini‑batches that the model can consume.

### Why a pipeline matters
Think of the pipeline as a factory line. If one station is slow or buggy, the whole line stalls. A well‑designed pipeline ensures that:
- **Speed**: Data is pre‑processed once and cached, so training loops run fast.
- **Reproducibility**: Fixed random seeds and deterministic shuffling mean you can hit the same data split every run.
- **Scalability**: Streaming large corpora keeps memory usage low, allowing you to train on datasets that would otherwise not fit in RAM.

### Key terms explained
- **Tokenizer**: Turns raw text into a list of token IDs.
- **Dataset**: A collection of examples (here, tokenized sequences) that can be iterated over.
- **Collator**: A function that pads a batch of sequences to the same length.
- **Streaming**: Loading data on‑the‑fly from disk or the internet, rather than loading everything into memory.
- **Cache**: A local copy of processed data that speeds up subsequent runs.
- **Seed**: A number that initializes random number generators to make shuffling deterministic.

### Trade‑offs
| Decision | Pros | Cons |
|----------|------|------|
| **Full‑text tokenization vs. chunking** | Keeps context intact; fewer padding tokens | Requires more memory; longer sequences may hit model limits |
| **Caching vs. on‑the‑fly processing** | Faster subsequent runs | Disk space usage; stale cache if preprocessing changes |
| **Streaming vs. loading into RAM** | Handles arbitrarily large datasets | Slightly slower per‑epoch throughput due to I/O |
| **Deterministic shuffling vs. random shuffling** | Reproducible experiments | May introduce subtle bias if the same order is always used |

Choosing the right balance depends on your compute budget, dataset size, and the level of reproducibility you need. In the code below we’ll use a streaming approach with deterministic shuffling and cache the processed dataset to disk.



In [ ]:
# Cell 1: Load, clean, and tokenize a streaming dataset
# ---------------------------------------------------------------
# Imports
import os
import random
import torch
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer

# 1️⃣ Set reproducibility seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

# 2️⃣ Choose a public dataset (e.g., WikiText-2) or a local text file
# For illustration we use the "wikitext" dataset; replace with your own path if needed
DATASET_NAME = "wikitext"
DATASET_CONFIG = "wikitext-2-raw-v1"

print(f"Loading {DATASET_NAME} ({DATASET_CONFIG}) in streaming mode…")
raw_ds = load_dataset(DATASET_NAME, DATASET_CONFIG, split="train", streaming=True)

# 3️⃣ Define a simple cleaning function

def clean_text(example):
    """Strip leading/trailing whitespace and collapse multiple spaces."""
    text = example["text"].strip()
    text = " ".join(text.split())  # collapse whitespace
    return {"text": text}

# 4️⃣ Apply cleaning and tokenization in a single pass
TOKENIZER_NAME = "gpt-oss-20b"
print("Loading tokenizer…")
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

MAX_LENGTH = 512  # truncate long sequences to fit model context

def tokenize(example):
    """Tokenize and truncate to MAX_LENGTH."""
    tokens = tokenizer(example["text"], truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
    return {"input_ids": tokens["input_ids"].squeeze(0)}

# 5️⃣ Stream, clean, and tokenize
print("Processing stream… (this may take a few minutes)")
processed = raw_ds.map(clean_text).map(tokenize, batched=False)

# 6️⃣ Convert to a Dataset (not streaming) and cache to disk
CACHE_DIR = "./cached_wikitext"
print(f"Caching processed dataset to {CACHE_DIR}…")
processed = processed.with_format("torch")  # ensure torch tensors
processed.save_to_disk(CACHE_DIR)
print("Dataset cached successfully.")


In [ ]:
# Cell 2: Load cached dataset, split, shuffle, and create a DataLoader
# ---------------------------------------------------------------
import torch
from torch.utils.data import DataLoader
from datasets import load_from_disk
from transformers import DataCollatorForLanguageModeling

# 1️⃣ Load the cached dataset
print("Loading cached dataset…")
cached_ds = load_from_disk("./cached_wikitext")
print(f"Total examples: {len(cached_ds)}")

# 2️⃣ Split into train/validation (80/20) with deterministic shuffling
print("Splitting dataset…")
train_val = cached_ds.train_test_split(test_size=0.2, seed=SEED)
train_ds = train_val["train"]
val_ds = train_val["test"]
print(f"Train size: {len(train_ds)}, Validation size: {len(val_ds)}")

# 3️⃣ Create a collator that pads to the longest sequence in the batch
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 4️⃣ Build DataLoaders
BATCH_SIZE = 8
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collator)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collator)

print("DataLoaders ready. Example batch shape:")
for batch in train_loader:
    print(batch["input_ids"].shape)  # (batch, seq_len)
    break



## Step 5: Fine‑Tuning with Parameter‑Efficient Tuning (PEFT)

Fine‑tuning a 20B‑parameter model on a single GPU is like trying to teach a giant elephant to do a tiny trick – it’s doable but expensive. Parameter‑Efficient Tuning (PEFT) tricks the elephant into learning by only moving a few *tweak‑points* instead of reshaping its entire body. In practice, we freeze the bulk of GPT‑Oss‑20B and insert lightweight adapters (e.g., LoRA) that learn the task‑specific signal. The result is a model that behaves like a fully fine‑tuned one but costs a fraction of the memory and compute.

### Why PEFT?
- **Memory savings**: Only a few thousand extra parameters are trained, so a single 24 GB GPU can handle the whole fine‑tune.
- **Speed**: Training time drops because gradients flow through a tiny network.
- **Flexibility**: You can swap adapters for different tasks without touching the base weights.
- **Safety**: The frozen backbone preserves the original knowledge, reducing catastrophic forgetting.

### Key terms & trade‑offs
| Term | What it means | Why it matters | Trade‑off |
|------|---------------|----------------|-----------|
| **LoRA (Low‑Rank Adaptation)** | Adds rank‑`r` matrices to the weight matrices of a transformer layer. | Keeps the number of trainable parameters low while still capturing task‑specific patterns. | Choosing a very low rank may hurt performance; a high rank increases memory again. |
| **Adapter** | A small neural module inserted between layers. | Allows fine‑tuning without touching the original weights. | Adds a tiny inference overhead. |
| **Freeze** | Keep the original weights fixed during training. | Prevents over‑fitting and saves memory. | Limits the model’s ability to fully adapt to extreme domain shifts. |
| **PEFT library** | Hugging Face’s `peft` package that implements LoRA, Prefix, and other adapters. | Provides a clean API to wrap any Hugging Face model. | Requires an extra dependency and a bit of boilerplate. |

The trade‑off is essentially *precision vs. efficiency*. If your downstream task is very different from the pre‑training data, a full fine‑tune might still win. For most domain‑adaptation scenarios, LoRA gives you a sweet spot.

### What we’ll do in code
1. Load GPT‑Oss‑20B and its tokenizer.
2. Wrap the model with a LoRA adapter (rank = 8, alpha = 32).
3. Prepare a small synthetic dataset (for demo purposes) – in practice you’d use your domain corpus.
4. Set up `TrainingArguments` and a `Trainer` that only updates the LoRA weights.
5. Run a quick training loop and inspect the adapter weights.

All code cells are under 30 lines and include comments for clarity.



In [ ]:
# Cell 1: Install PEFT (if not already installed) and import libraries
# ---------------------------------------------------------------
import subprocess, sys

try:
    import peft
except ImportError:
    cmd = [sys.executable, "-m", "pip", "install", "peft==0.5.0", "--quiet"]
    try:
        subprocess.check_call(cmd)
    except Exception as exc:
        if IN_COLAB:
            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
            if packages:
                try:
                    import IPython
                    ip = IPython.get_ipython()
                    if ip is not None:
                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                    else:
                        import subprocess as _subprocess
                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                except Exception as colab_exc:
                    print('⚠️ Colab pip fallback failed:', colab_exc)
                    raise
            else:
                print('No packages specified for pip install; skipping fallback')
        else:
            raise
    import peft

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Set reproducibility
SEED = 123
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Load base model and tokenizer
MODEL_NAME = "gpt-oss-20b"
print("Loading tokenizer…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("Loading base model (FP16)…")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda:0" if torch.cuda.is_available() else "cpu"
)

# Define LoRA configuration
lora_cfg = LoraConfig(
    r=8,          # rank of the low‑rank matrices
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # apply to query & value projections
    lora_dropout=0.05,
    bias="none"
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_cfg)
print("LoRA adapters added – trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))



In [ ]:
# Cell 2: Create a tiny synthetic dataset for demonstration
# ---------------------------------------------------------------
from datasets import Dataset

# Simple sentences – replace with your real data
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Deep learning models can be fine‑tuned efficiently.",
    "Parameter‑efficient tuning saves memory and time.",
    "LoRA adds low‑rank adapters to transformer layers.",
    "GPT‑Oss‑20B is a large language model.",
]

# Tokenize and create a Dataset
tokenized = tokenizer(sentences, truncation=True, max_length=128, return_tensors="pt")
train_ds = Dataset.from_dict({"input_ids": tokenized["input_ids"], "attention_mask": tokenized["attention_mask"]})

# Define training arguments – small epoch count for demo
training_args = TrainingArguments(
    output_dir="./peft_output",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    logging_steps=1,
    save_strategy="no",
    fp16=True,
    seed=SEED,
)

# Trainer that only updates LoRA weights
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
    data_collator=lambda data: {
        "input_ids": torch.stack([f["input_ids"] for f in data]),
        "attention_mask": torch.stack([f["attention_mask"] for f in data]),
    },
)

print("Starting fine‑tune…")
trainer.train()
print("Training finished. Adapter weights are now updated.")



## Step 6: Distributed Training Using DeepSpeed

### Why go distributed?
Imagine you’re baking a huge cake that needs 20 B layers of frosting. One oven can only fit a few layers at a time, so you split the job across many ovens and let them work in parallel. Distributed training does the same thing for a gigantic language model: it splits the model and data across several GPUs so that each GPU does a piece of the work. The result is a *faster* training loop and a *smaller* memory footprint per device.

### DeepSpeed in a nutshell
DeepSpeed is a library that makes distributed training efficient and easy. It adds three main ingredients:

1. **ZeRO (Zero Redundancy Optimizer)** – a memory‑optimization technique that removes duplicate copies of model parameters, gradients, and optimizer states across GPUs.
2. **Automatic Mixed‑Precision (AMP)** – runs most operations in FP16 or BF16 to cut memory usage and speed up compute.
3. **Dynamic Loss Scaling** – keeps gradients stable when using low‑precision arithmetic.

Think of ZeRO as a *shared pantry* where each kitchen only keeps the ingredients it needs, instead of each kitchen storing a full set of spices.

### Key terms & trade‑offs
| Term | What it means | Why it matters | Trade‑off |
|------|---------------|----------------|-----------|
| **Distributed Data Parallel (DDP)** | Each GPU holds a copy of the model and processes a mini‑batch; gradients are averaged across GPUs. | Enables parallelism across GPUs. | Requires careful synchronization; communication overhead can grow with GPU count. |
| **ZeRO Stage 1** | Shards optimizer states across GPUs. | Reduces memory by ~1/num_gpus. | Still keeps full parameters on each GPU. |
| **ZeRO Stage 2** | Shards optimizer states *and* gradients. | Further memory savings, enabling larger batch sizes. | Slightly more communication during backward pass. |
| **ZeRO Stage 3** | Shards parameters, optimizer states, and gradients. | Max memory efficiency; can train models that otherwise would not fit. | Highest communication cost; best suited for very large models. |
| **AMP** | Uses FP16/BF16 arithmetic. | Cuts memory and speeds up GPU kernels. | Requires loss scaling to avoid underflow. |
| **Dynamic Loss Scaling** | Adjusts the scaling factor during training. | Keeps gradients in a safe range. | Adds a small runtime overhead. |

### Rationale for DeepSpeed + PEFT
When fine‑tuning GPT‑Oss‑20B with LoRA adapters, the majority of the 20 B parameters stay frozen. However, the *optimizer state* for the LoRA weights still needs to be stored on each GPU. ZeRO‑2 or ZeRO‑3 lets us shard that tiny state across GPUs, freeing up memory for larger batch sizes or higher‑rank adapters. AMP ensures we can keep the model in FP16 without sacrificing stability.

### What we’ll do in code
1. Create a minimal `deepspeed_config.json` that enables ZeRO‑2, AMP, and dynamic loss scaling.
2. Write a lightweight training script that uses Hugging Face `Accelerate` to launch DeepSpeed.
3. Show the command line to start training on 4 GPUs.

All code cells are under 30 lines and include comments for clarity.



In [ ]:
# Cell 1: Create a DeepSpeed config file (deepspeed_config.json)
# ---------------------------------------------------------------
import json
config = {
    "train_batch_size": 8,          # global batch size (per GPU * num_gpus)
    "gradient_accumulation_steps": 1,
    "fp16": {"enabled": True},    # enable automatic mixed precision
    "zero_optimization": {
        "stage": 2,                # ZeRO‑2: shard optimizer states & gradients
        "allgather_partitions": True,
        "reduce_scatter": True,
        "contiguous_gradients": True
    },
    "optimizer": {"type": "AdamW", "params": {"lr": 5e-5}},
    "gradient_clipping": 1.0,
    "loss_scale": 0,               # dynamic loss scaling
    "zero_allow_untested_optimizer": true
}
with open("deepspeed_config.json", "w") as f:
    json.dump(config, f, indent=4)
print("DeepSpeed config written to deepspeed_config.json")



In [ ]:
# Cell 2: Minimal training script using Accelerate + DeepSpeed
# ---------------------------------------------------------------
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig

# 1️⃣ Setup accelerator (will detect DeepSpeed if available)
accelerator = Accelerator()

# 2️⃣ Load base model & tokenizer
MODEL_NAME = "gpt-oss-20b"
print("Loading tokenizer…")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("Loading base model (FP16)…")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cuda:0" if torch.cuda.is_available() else "cpu"
)

# 3️⃣ Wrap with LoRA (rank 8) – only these weights will be trained
lora_cfg = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")
model = get_peft_model(model, lora_cfg)
print("LoRA adapters added – trainable params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

# 4️⃣ Prepare a tiny synthetic dataset (replace with real data)
from datasets import Dataset
sentences = ["DeepSpeed makes training large models fast.", "LoRA adapters keep memory low."]
enc = tokenizer(sentences, truncation=True, max_length=128, return_tensors="pt")
train_ds = Dataset.from_dict({"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"]})

# 5️⃣ Training arguments – point to the DeepSpeed config
training_args = TrainingArguments(
    output_dir="./ds_output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    logging_steps=1,
    fp16=True,
    deepspeed="deepspeed_config.json",
    seed=42
)

# 6️⃣ Build Trainer (Accelerator will wrap it for DeepSpeed)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
    data_collator=lambda data: {
        "input_ids": torch.stack([f["input_ids"] for f in data]),
        "attention_mask": torch.stack([f["attention_mask"] for f in data])
    }
)

# 7️⃣ Launch training – Accelerator handles DeepSpeed launch
print("Starting distributed training…")
trainer.train()
print("Training finished. LoRA weights updated.")



In [ ]:
# Cell 3: Command line to launch training on 4 GPUs
# ---------------------------------------------------------------
# Save this script as train_ds.py and run:
#   deepspeed --num_gpus=4 train_ds.py
#
# If you prefer Accelerate’s launcher:
#   accelerate launch --num_gpus=4 train_ds.py
#
# The `deepspeed_config.json` file created earlier will be automatically picked up.
print("Use the above command to start distributed training.")



## Knowledge Check (Interactive)

Use the widgets below to select an answer and click Grade to see feedback.


In [ ]:
# MCQ helper (ipywidgets)
import ipywidgets as widgets
from IPython.display import display, Markdown

def render_mcq(question, options, correct_index, explanation):
    # Use (label, value) so rb.value is the numeric index
    rb = widgets.RadioButtons(options=[(f'{chr(65+i)}. '+opt, i) for i,opt in enumerate(options)], description='')
    grade_btn = widgets.Button(description='Grade', button_style='primary')
    feedback = widgets.HTML(value='')
    def on_grade(_):
        sel = rb.value
        if sel is None:
            feedback.value = '<p>⚠️ Please select an option.</p>'
            return
        if sel == correct_index:
            feedback.value = '<p>✅ Correct!</p>'
        else:
            feedback.value = f'<p>❌ Incorrect. Correct answer is {chr(65+correct_index)}.</p>'
        feedback.value += f'<div><em>Explanation:</em> {explanation}</div>'
    grade_btn.on_click(on_grade)
    display(Markdown('### '+question))
    display(rb)
    display(grade_btn)
    display(feedback)


In [ ]:
render_mcq("Which of the following is NOT a benefit of using DeepSpeed for training large language models?", ["Memory optimization through ZeRO stages","Automatic mixed‑precision training","Built‑in support for LoRA parameter‑efficient tuning","Dynamic loss scaling for numerical stability"], 2, "DeepSpeed provides memory optimization, mixed‑precision, and loss scaling, but LoRA tuning is a separate library (PEFT) that can be integrated with DeepSpeed.")


In [ ]:
render_mcq("What is the primary advantage of parameter‑efficient tuning over full fine‑tuning?", ["Higher training speed due to fewer parameters","Better generalization on unseen data","Reduced GPU memory footprint during training","Elimination of the need for a validation set"], 2, "PEFT methods freeze most of the model, training only a small set of adapters, which dramatically reduces memory usage and speeds up training while maintaining performance.")


## 🔧 Troubleshooting Guide

### Common Issues:

1. **Out of Memory Error**
   - Enable GPU: Runtime → Change runtime type → GPU
   - Restart runtime if needed

2. **Package Installation Issues**
   - Restart runtime after installing packages
   - Use `!pip install -q` for quiet installation

3. **Model Loading Fails**
   - Check internet connection
   - Verify authentication tokens
   - Try CPU-only mode if GPU fails
